In [9]:
import os
import ast
import pandas as pd
import psycopg2
from collections import defaultdict
from sqlalchemy import create_engine
from urllib.parse import quote_plus

# researchers_path = os.path.join(os.getcwd(), "datasets", "users", 'researchers.xlsx') 
# articles_path = os.path.join(os.getcwd(), "datasets", "articles", "excels_CS", "articles.xlsx")
# researchers_file_path = os.path.join(os.getcwd(), "datasets", "users", 'researchers.xlsx')

In [8]:
import sys
!{sys.executable} -m pip install sqlalchemy
!{sys.executable} -m pip install psycopg2-binary

  Using cached SQLAlchemy-2.0.23-cp310-cp310-win_amd64.whl (2.1 MB)
  Using cached greenlet-3.0.1-cp310-cp310-win_amd64.whl (287 kB)


You should consider upgrading via the 'c:\Users\bianca\Desktop\AOI\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\Users\bianca\Desktop\AOI\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [28]:
# Establish PostgreSQL connection details
db_user = 'postgres'
db_password = 'p@$$w0rd!'
db_host = 'localhost'
db_port = '5432'
db_name = 'AOI'

# Format the password for the connection string
# This escapes special characters in the password
password = quote_plus(db_password)

# Create a connection to the PostgreSQL database
conn = psycopg2.connect(
    dbname=db_name,
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port
)

# Create a SQLAlchemy engine
engine = create_engine(f'postgresql://{db_user}:{password}@{db_host}:{db_port}/{db_name}')

# Fetch data from PostgreSQL tables into pandas DataFrames
researchers_query = 'SELECT "Researcher ID", "Full Name", "Expertise", "Appreciated" FROM researchers_table'
articles_query = 'SELECT id, title, authors, url, subject_split FROM articles_table'

researchers_df = pd.read_sql(researchers_query, engine)
articles_df = pd.read_sql(articles_query, engine)

researchers_df.head()


,Researcher ID,Full Name,Expertise,Appreciated
0,L-6699-2016,"DEACONESCU, Razvan","['Systems', 'Security']","arXiv:2310.03598, arXiv:2310.03994, arXiv:2310.12254, arXiv:2310.11651, arXiv:2310.10653, arXiv:2310.18820, arXiv:2311.05498, arXiv:2311.05462, arXiv:2311.05037, arXiv:2311.02378"
1,AAG-9392-2021,"APOSTOL, Elena Simona","['Distributed systems', 'IT security', 'Parallel algorithms', 'Machine learning', 'Genetic algorithms']","arXiv:2310.02113, arXiv:2310.05269, arXiv:2310.11730, arXiv:2310.09665, arXiv:2310.13424, arXiv:2311.02100, arXiv:2310.01689, arXiv:2310.01676, arXiv:2310.06338, arXiv:2310.04975"
2,JCE-1061-2023,"CARABAS, Costin","['Computer Science', 'Software Engineering', 'Telecommunications', 'Automation & Control Systems']","arXiv:2310.03736, arXiv:2310.03702, arXiv:2310.03655, arXiv:2310.03528, arXiv:2310.03501, arXiv:2310.03475, arXiv:2310.03441, arXiv:2310.03178, arXiv:2310.03159, arXiv:2310.03105"
3,AAY-5210-2020,"CHIRU, Costin Gabriel","['NLP', 'Machine learning', 'Artificial intelligence', 'Algorithms', 'Logical design', 'Teaching technology-enhanced learning', 'Group knowledge', 'Virtual teams', 'Collaboration']","arXiv:2310.02357, arXiv:2310.14261, arXiv:2310.01551, arXiv:2310.00526, arXiv:2310.05309, arXiv:2310.19647, arXiv:2311.05511, arXiv:2311.01927, arXiv:2310.04218, arXiv:2310.04425"
4,C-5751-2009,"CIRTOAJE, Cristina","['Liquid crystal', 'Liquid crystals, polarised microscopy, magneto-optic and electro-optic properties', 'Carbon based nanomaterials', 'Nanoscience', 'Nanoparticles', 'Magnetic nanomaterials', 'Carbon based nanomaterials', 'Quantum dots', 'Physics', 'Materials Science', 'Mathematics', 'Science & Technology - Other Topics', 'Optics']","arXiv:2310.04022, arXiv:2310.10524, arXiv:2310.03557, arXiv:2310.03339, arXiv:2310.03254, arXiv:2310.03193, arXiv:2310.01283, arXiv:2310.00394, arXiv:2310.01046, arXiv:2310.00267"


In [94]:
# researchers_df = pd.read_excel(researchers_path, sheet_name="Sheet")
# articles_df = pd.read_excel(articles_path, sheet_name="Sheet1")

In [11]:
def assign_weights(interests):
    num_interests = len(interests)
  
    weights = []
    weight = 0.9

    for i in range(num_interests):
        weights.append((interests[i], round(weight, 3)))
        weight *= 0.8
    
    return weights

In [12]:
def compute_relevance(article_subjects, researcher_interests):
    researcher_interests_dict = dict(researcher_interests)
    matched_subjects = []
    
    # print("Researcher interests dict:", researcher_interests_dict)
    
    relevance_score = 0
    
    for subject in article_subjects:
        # print("Current article subj:", subject)
        for interest, weight in researcher_interests_dict.items():
            # print("Current researcher interest:", interest)
            if (interest.lower() == subject.lower()) or (subject.lower() == "security" and interest.lower() == "it security"):
                # print(f"match found: {interest} in {subject}")
                relevance_score += weight
                matched_subjects.append(interest)
    

    # print("Article Subjects:", article_subjects)
    # # print("Relevance Score:", relevance_score)
    # print()
    
    return relevance_score, matched_subjects


In [17]:
articles_recommended_to_researchers = defaultdict(set)
top_articles = defaultdict()

for index, researcher in researchers_df.iterrows():

    interests_as_string = researcher['Expertise']
    interests = ast.literal_eval(interests_as_string)
    num_interests = len(interests)

    weighted_interests = assign_weights(interests)
    # print(f"Weighted interests for {researcher['Full Name']}:", weighted_interests)

    articles_relevance = []

    for index_a, article in articles_df.iterrows():
        article_id = article['id']
        # print(f"Article id is {article_id}")
        article_subjects_as_string = article['subject_split']
        article_subjects = ast.literal_eval(article_subjects_as_string)
        # compute relevance score for current article
        relevance_score, matched_subjects = compute_relevance(article_subjects, weighted_interests)
        articles_relevance.append((article, relevance_score, matched_subjects))

    # Debugging..
    # print articles and their scores for  current researcher
    # print(f"\nArticles relevance for {researcher['Full Name']}:")
    
    # for article, relevance_score in articles_relevance:
    #     print(f"Article ID: {article['id']} | Title: {article['title']} | Relevance Score: {relevance_score}")
    #     print('*' * 60)

    articles_relevance.sort(key=lambda x: x[1], reverse=True)
    # select top 10 fits
    researcher_id = researcher['Researcher ID']
    top_articles[researcher_id] = articles_relevance[:10]

    for article, _, _ in top_articles[researcher_id]:
        article_id = article['id']
        articles_recommended_to_researchers[article_id].add(researcher['Full Name'])

    # select only common recomm
    articles_recommended_to_multiple_researchers = {article_id: researchers 
                                                    for article_id, researchers in articles_recommended_to_researchers.items() 
                                                    if len(researchers) >= 2}

    
    print(f"\nTop 10 recommendations according to interests for {researcher['Full Name']}:")
    for article, relevance_score, matched_subjects in top_articles[researcher_id]:
        print(f"ID: {article['id']} | URL: {article['url']} | Title: {article['title']} | Relevance Score: {relevance_score} | Matched Subjects: {matched_subjects}")
    



Top 10 recommendations according to interests for APOSTOL, Elena Simona:
ID: arXiv:2310.02113 | URL: https://arxiv.org/pdf/2310.02113.pdf | Title: FLEDGE: Ledger-based Federated Learning Resilient to Inference and  Backdoor Attacks | Relevance Score: 2.081 | Matched Subjects: ['Machine learning', 'Distributed systems', 'IT security']
ID: arXiv:2310.05269 | URL: https://arxiv.org/pdf/2310.05269.pdf | Title: Federated Learning: A Cutting-Edge Survey of the Latest Advancements and  Applications | Relevance Score: 2.081 | Matched Subjects: ['Machine learning', 'Distributed systems', 'IT security']
ID: arXiv:2310.11730 | URL: https://arxiv.org/pdf/2310.11730.pdf | Title: Federated Heterogeneous Graph Neural Network for Privacy-preserving  Recommendation | Relevance Score: 2.081 | Matched Subjects: ['Machine learning', 'Distributed systems', 'IT security']
ID: arXiv:2310.09665 | URL: https://arxiv.org/pdf/2310.09665.pdf | Title: A Blockchain-empowered Multi-Aggregator Federated Learning Arc

### Save common interests

In [21]:
# common interests (helper for later debugging?)
with open('articles_recommended_to_multiple_researchers.txt', 'w') as file:
    for article_id, researchers in articles_recommended_to_multiple_researchers.items():
        file.write(f"Article ID: {article_id}\n")
        file.write("Recommended to Researchers:\n")
        for researcher in researchers:
            file.write(f"- {researcher}\n")
        file.write("\n")

#### Update Appreciated column

In [27]:
# pd.set_option('display.max_colwidth', None)

researchers_df['Appreciated'] = researchers_df['Appreciated'].astype('object')

# for index, researcher in researchers_df.iterrows():
#     researcher_id = researcher['Researcher ID']
#     researcher_name = researcher['Full Name']
#     # ids of recommended articles
#     recommended_articles = [article['id'] for article, _, _ in top_articles[researcher_id] if researcher_name in articles_recommended_to_researchers[article['id']]]
#     recommended_articles_str = ', '.join(recommended_articles)
#     researchers_df.at[index, 'Appreciated'] = recommended_articles_str


# researchers_df.to_excel(researchers_file_path, index=False)

# Create a connection to the PostgreSQL database
conn = psycopg2.connect(
    dbname=db_name,
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port
)
# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Loop through each row in researchers_df and update the 'Appreciated' column in the 'researchers_table'
for index, researcher in researchers_df.iterrows():
    researcher_id = researcher['Researcher ID']
    researcher_name = researcher['Full Name']
    recommended_articles = [article['id'] for article, _, _ in top_articles[researcher_id] if researcher_name in articles_recommended_to_researchers[article['id']]]
    recommended_articles_str = ', '.join(recommended_articles)
    
    # Execute an SQL UPDATE statement
    update_query = f"UPDATE researchers_table SET \"Appreciated\" = '{recommended_articles_str}' WHERE \"Researcher ID\" = '{researcher_id}'"
    cursor.execute(update_query)

# Commit the changes to the database
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()